# SST: Interdecadal Pacific Oscillation / Interdecadal Pacific Oscillation

## IPO
Henley at al. (2015): _A Tripole Index for the Interdecadal Pacific Oscillation_:
>The following method is used to compute the Tripole Index, based on monthly global SST data:
> 1. Subtract the monthly climatology from each SST grid cell to remove the seasonal cycle and compute the monthly mean SST anomalies (SSTAi) in each of the three TPI regions using a chosen base period (1971– 2000 used here).
> 2. Compute the unfiltered TPI as:
> $ TPI = TPI2 - (TPI1+TPI3)/2$
> 3. Apply a 13-year Chebyshev low-pass filter to obtain the filtered version of the index (filtered TPI).

In [ ]:
import sys
sys.path.append("..")
import scipy as sp
import numpy as np
import xarray as xr
import seaborn as sns
import cmocean
import cartopy
import cartopy.crs as ccrs
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2
%aimport - numpy - scipy - matplotlib.pyplot

In [ ]:
from OHC import t2da, t2ds
from SST import SST_index, EOF_SST_analysis
from maps import map_robinson, map_eq_earth, rect_polygon
from grid import find_array_idx
from paths import path_results, path_samoc, file_ex_ocn_ctrl, file_ex_ocn_rect
from regions import boolean_mask, SOM_area, global_ocean, gl_ocean_rect, NPacific_mask_rect, TPI_masks
from plotting import shifted_color_map, discrete_cmap
from timeseries import IterateOutputCESM, lowpass, chebychev
from xr_DataArrays import xr_AREA
from xr_regression import xr_lintrend, xr_linear_trends_2D, xr_linear_trend, ocn_field_regression, lag_linregress_3D

# Tripole index __TPI__
### raw SST indices of TPI1-3
running the `IPO_TPI` function of `SST.py`  via the terminal or

`%run ../SST.py IPO {run}`

results in raw monthly timeseries of the three TPI SST indices (takes 30 minutes for rcp run)

In [ ]:
runs = ['ctrl', 'rcp', 'lpd', 'lpi']

In [ ]:
TPI1_ctrl = xr.open_dataarray(f'{path_samoc}/SST/TPI1_ctrl.nc', decode_times=False)
TPI1_rcp  = xr.open_dataarray(f'{path_samoc}/SST/TPI1_rcp.nc' , decode_times=False)
TPI1_lpd  = xr.open_dataarray(f'{path_samoc}/SST/TPI1_lpd.nc' , decode_times=False)
TPI1_lpi  = xr.open_dataarray(f'{path_samoc}/SST/TPI1_lpi.nc' , decode_times=False)
TPI2_ctrl = xr.open_dataarray(f'{path_samoc}/SST/TPI2_ctrl.nc', decode_times=False)
TPI2_rcp  = xr.open_dataarray(f'{path_samoc}/SST/TPI2_rcp.nc' , decode_times=False)
TPI2_lpd  = xr.open_dataarray(f'{path_samoc}/SST/TPI2_lpd.nc' , decode_times=False)
TPI2_lpi  = xr.open_dataarray(f'{path_samoc}/SST/TPI2_lpi.nc' , decode_times=False)
TPI3_ctrl = xr.open_dataarray(f'{path_samoc}/SST/TPI3_ctrl.nc', decode_times=False)
TPI3_rcp  = xr.open_dataarray(f'{path_samoc}/SST/TPI3_rcp.nc' , decode_times=False)
TPI3_lpd  = xr.open_dataarray(f'{path_samoc}/SST/TPI3_lpd.nc' , decode_times=False)
TPI3_lpi  = xr.open_dataarray(f'{path_samoc}/SST/TPI3_lpi.nc' , decode_times=False)

In [ ]:
TPI_list_ctrl = [TPI1_ctrl, TPI2_ctrl, TPI3_ctrl]
TPI_list_rcp  = [TPI1_rcp , TPI2_rcp , TPI3_rcp ]
TPI_list_lpd  = [TPI1_lpd , TPI2_lpd , TPI3_lpd ]
TPI_list_lpi  = [TPI1_lpi , TPI2_lpi , TPI3_lpi ]
TPIs = [TPI_list_ctrl, TPI_list_rcp, TPI_list_lpd, TPI_list_lpi]

### detrending

In [ ]:
def plot_fits(ts, ax, color, label):
    lf = np.polyfit(ts.time, ts.values, 1)
    qf = np.polyfit(ts.time, ts.values, 2)
    ax[0].plot(ts.time/365, lf[0]*ts.time                    -  lf[0]*ts.time[0]                       , c=color, label=label)
    ax[1].plot(ts.time/365, qf[0]*ts.time**2 + qf[1]*ts.time - (qf[0]*ts.time[0]**2 + qf[1]*ts.time[0]), c=color, label=label)
    plt.tight_layout()

In [ ]:
f,ax = plt.subplots(2,4, figsize=(12,5), sharex='col')
for k, TPI_list in enumerate(TPIs):
    for i, TPI in enumerate(TPI_list):
        plot_fits(TPI, ax[:,k], f'C{i}', ['TPI1', 'TPI2', 'TPI3'][i])
    ax[0,k].set_title(runs[k])
    ax[1,k].set_xlabel('time [years]')
ax[0,0].legend()
ax[0,0].set_ylabel('linear fit')
ax[1,0].set_ylabel('quad. fit')
f.align_labels()
plt.tight_layout()

I will take the average linear trend to subtract from all three TPI timeseries, to stay physically consistent. Luckily, TPI1 and TPI3 trends are enclose TPI2, so that the error due to this approximation should reduce.

In [ ]:
def detrend_TPI_lists(TPI_list):
    lf = np.zeros((3,2))
    TPI_dts = []
    for i, TPI in enumerate(TPI_list):
        lf[i,:] = np.polyfit(TPI.time, TPI.values, 1)
#         plt.plot(TPI.time/365 - TPI.time[0]/365, lf[i,0]*TPI.time - lf[i,0]*TPI.time[0])
        if i==2:
            lf_avg = lf[:,0].mean()
#             plt.plot(TPI.time/365 - TPI.time[0]/365, lf_avg*TPI.time - lf_avg*TPI.time[0])
    for i, TPI in enumerate(TPI_list):
        TPI_dt = TPI - TPI.time*lf_avg
        TPI_dts.append(TPI_dt - TPI_dt.mean())
    return TPI_dts

### deseasonalize 

In [ ]:
def deseasonalize_TPI_dts(TPI_dts):
    seasons = np.zeros((3,12))
    TPI_dtdss = []
    for i, TPI in enumerate(TPI_dts):
        TPI_dtds = TPI.copy()
        for j in range(12):
            TPI_dtds[j::12] -= TPI[j::12].mean()
        TPI_dtdss.append(TPI_dtds)
    return TPI_dtdss

### calculate final tripole IPO index 
$$ TPI = TPI2 - \frac{TPI1+TPI3}{2}$$

In [ ]:
plt.axhline(0, c='k', lw=.5)
for i, TPI_list in enumerate(TPIs):
    TPI_dts   = detrend_TPI_lists(TPI_list)
    TPI_dtdss = deseasonalize_TPI_dts(TPI_dts)
    for j in range(3):
#         plt.plot(TPI_dts[i]  .time/365 - TPI_dts[i]  .time[0]/365, TPI_dts[i]  , alpha=.1)
        plt.plot(TPI_dtdss[j].time/365 - TPI_dtdss[j].time[0]/365, TPI_dtdss[j], alpha=.1)
    TPI_dtds = TPI_dtdss[1] - (TPI_dtdss[0] + TPI_dtdss[2])/2
    TPI_dtds.to_netcdf(f'{path_results}/SST/TPI_monthly_{runs[i]}.nc')

### create yearly TPI index

In [ ]:
# TPI_monthly_ctrl = xr.open_dataarray(f'{path_results}/SST/TPI_monthly_ctrl.nc', decode_times=False)
# TPI_monthly_rcp  = xr.open_dataarray(f'{path_results}/SST/TPI_monthly_rcp.nc' , decode_times=False)
# TPI_monthly_lpd  = xr.open_dataarray(f'{path_results}/SST/TPI_monthly_lpd.nc' , decode_times=False)
# TPI_monthly_lpi  = xr.open_dataarray(f'{path_results}/SST/TPI_monthly_lpi.nc' , decode_times=False)

In [ ]:
# def yearly_signal(TPI):
#     times, values = [], []
#     for k,g in TPI.groupby_bins('time', xr.concat([TPI.time[::12], TPI.time[-1]], dim='time')):
#         times.append(k.left)
#         values.append(float(g.mean()))
#     da = xr.DataArray(data=values, coords={'time': times}, dims=('time'))
#     return da

In [ ]:
# yearly_signal(TPI_monthly_ctrl).to_netcdf(f'{path_results}/SST/TPI_ctrl.nc')
# yearly_signal(TPI_monthly_rcp ).to_netcdf(f'{path_results}/SST/TPI_rcp.nc')
# yearly_signal(TPI_monthly_lpd ).to_netcdf(f'{path_results}/SST/TPI_lpd.nc')
# yearly_signal(TPI_monthly_lpi ).to_netcdf(f'{path_results}/SST/TPI_lpi.nc')

# Analysis of TPI

In [ ]:
TPI_ctrl = xr.open_dataarray(f'{path_results}/SST/TPI_ctrl.nc', decode_times=False)
TPI_rcp  = xr.open_dataarray(f'{path_results}/SST/TPI_rcp.nc' , decode_times=False)
TPI_lpd  = xr.open_dataarray(f'{path_results}/SST/TPI_lpd.nc' , decode_times=False)
TPI_lpi  = xr.open_dataarray(f'{path_results}/SST/TPI_lpi.nc' , decode_times=False)

In [ ]:
plt.figure(figsize=(8,5))
plt.axhline(0, c='k', lw=.5)
# plt.plot(TPI_dtds.time/365, TPI_dtds.rolling(time=12, center=True).mean(), alpha=.2)
plt.plot(TPI_ctrl.time/365, TPI_ctrl.rolling(time=100, center=True).mean(), label='100 year running mean')
plt.plot(TPI_ctrl.time/365, TPI_ctrl.rolling(time=13 , center=True).mean(), label='13 year running mean', alpha=.5)
plt.plot(TPI_ctrl.time/365, lowpass(TPI_ctrl    , 13)                     , label='13 year lowpass')
plt.plot(TPI_ctrl.time/365, chebychev(TPI_ctrl  , 13)                     , label='13 year chebychev')
# plt.plot(TPI_dtds.time/365, chebychev(TPI_dtds, 130*12), label='130 year chebychev')
plt.legend()
plt.tight_layout()

- the period of 13 years was given by Henley et al. (2015)
- the 'rp' value was set to 1 to minimize damping and ripple effects
- there are edge effects of parameter choices:
    * chose 'constant' padtype
    * of padlen N-1=5, there is a difference whether one uses monthly or yearly data (because he timeseries is either extended by 5 months or years, respectively)
    

In [ ]:
f, ax = plt.subplots(1,1,figsize=(12,5))
ax.tick_params(labelsize=14)
ax.axhline(0, c='k', lw=.5)

# ax.plot(TPI_ctrl.time/365+1850,           TPI_ctrl     , c='C0', ls='--', lw=.5)
# ax.plot(TPI_rcp .time/365+ 200,           TPI_rcp      , c='C1', ls='--', lw=.5)
# ax.plot(TPI_lpd .time/365+1350,           TPI_lpd      , c='C2', ls='--', lw=.5)
# ax.plot(TPI_lpi .time/365-1600,           TPI_lpi      , c='C3', ls='--', lw=.5)

L1, = ax.plot(TPI_ctrl.time/365+1850, chebychev(TPI_ctrl, 13), c='C0', ls='--', lw=1, label='chebychev 13 yr')
ax.plot(TPI_rcp .time/365+ 200, chebychev(TPI_rcp , 13), c='C1', ls='--', lw=1)
ax.plot(TPI_lpd .time/365+1350, chebychev(TPI_lpd , 13), c='C2', ls='--', lw=1)
ax.plot(TPI_lpi .time/365-1600, chebychev(TPI_lpi , 13), c='C3', ls='--', lw=1)

L2, = ax.plot(TPI_ctrl.time/365+1850, chebychev(TPI_ctrl, 40), c='C0', ls='-' , lw=2, label='lowpass 40 yr')
ax.plot(TPI_rcp .time/365+ 200, chebychev(TPI_rcp , 40), c='C1', ls='-' , lw=2)
ax.plot(TPI_lpd .time/365+1350, chebychev(TPI_lpd , 40), c='C2', ls='-' , lw=2)
ax.plot(TPI_lpi .time/365-1600, chebychev(TPI_lpi , 40), c='C3', ls='-' , lw=2)

L3, = ax.plot(TPI_ctrl.time/365+1850, xr_lintrend(TPI_ctrl), c='C0', ls='-.', lw=1.5, label='linear trend')
ax.plot(TPI_rcp .time/365+ 200, xr_lintrend(TPI_rcp ), c='C1', ls='-.', lw=1.5)
ax.plot(TPI_lpd .time/365+1350, xr_lintrend(TPI_lpd ), c='C2', ls='-.', lw=1.5)
ax.plot(TPI_lpi .time/365-1600, xr_lintrend(TPI_lpi ), c='C3', ls='-.', lw=1.5)

ax.plot([2050]*2, [-.15,.1], c='g', lw=.5)
ax.plot([2200]*2, [-.15,.1], c='g', lw=.5)

ax.text(1950, .12, 'CTRL'         , fontsize=16, color='C0')
ax.text(2200, .12, 'RCP'          , fontsize=16, color='C1')
ax.text(1500, .12, 'pres. day low', fontsize=16, color='C2')
ax.text(   0, .12, 'pre-ind. low' , fontsize=16, color='C3')

ax.legend(handles=[L1,L2,L3], loc=8, ncol=3, fontsize=14, frameon=False)

ax.set_ylabel('TPI index [K]', fontsize=16)
ax.set_xlabel('time [years]', fontsize=16)
ax.set_xticks(np.arange(0,2400,200))
f.tight_layout()
f.savefig(f'{path_results}/SST/TPI_index_overview.png')

## spectrogram

In [ ]:
TPIs = [TPI_ctrl, TPI_rcp, TPI_lpd, TPI_lpi]

f, ax = plt.subplots(1, 1, figsize=(8,5), sharey=True)
ax.set_ylabel('spectral power', fontsize=16)

for i in range (4):
    label= ['CTRL', 'RCP', 'LPD', 'LPI'][i]
    f, Pxx = sp.signal.welch(TPIs[i]-xr_lintrend(TPIs[i]), fs=1)
    ax.loglog(1/f, np.sqrt(Pxx), label=label)

ax.tick_params(labelsize=14)
ax.set_xlabel('period [yr]', fontsize=16)
ax.legend(fontsize=14, frameon=False)
    
plt.tight_layout()

## regression map

In [ ]:
plt.plot(SST_yrly_rcp[:,1500,3000]-SST_yrly_ctrl[:,1500,3000].mean(dim='time'))  # central North Pacific
plt.plot(SST_yrly_rcp[:,1800, 800]-SST_yrly_ctrl[:,1800, 800].mean(dim='time'))  # central North Atlantic
plt.plot(SST_yrly_rcp[:, 500,1300]-SST_yrly_ctrl[:, 500,1300].mean(dim='time'))  # Southern Ocean Sotuh of Africa
